<a href="https://colab.research.google.com/github/Pugianf/Big_Data_and_Public_Sector_I/blob/main/Aula_5_2021_10_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importações

In [ ]:
## Importando o que for necessário
import pandas as pd
import numpy as np
from scipy import stats
from zipfile import ZipFile
import statsmodels.api as sm
import statsmodels.stats.api as sms
from IPython.display import clear_output # limpa o output de uma célula

In [ ]:
## Modelos de painel
#! pip install linearmodels  # já atualiza o statsmodels (caso precise)
from linearmodels import PooledOLS, PanelOLS, RandomEffects
from linearmodels.panel import compare

## Para funções com modelos de painel, ver:
# https://bashtage.github.io/linearmodels/panel/introduction.html

In [ ]:
## Montando o Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## Definindo caminho no google drive on estão os dados
sCaminho = "/content/drive/MyDrive/Projetos/IDP/MBA/SetorPublicoI/código/Aula_Aplicação Paineis/"

# **Aplicação 1**

In [ ]:
## Lendo o dataset
# Descrição das variáveis: https://www.rdocumentation.org/packages/wooldridge/versions/1.4-1/topics/wagepan
sArquivo = f"{sCaminho}wagepan.dta"
df = pd.read_stata(sArquivo)

In [ ]:
## Visualizando o dataset
df.head(10)

# indivíduo: nr
# tempo: year

In [ ]:
# Vendo o tamanho da base
print(f"Linhas: {df.shape[0]}; Colunas: {df.shape[1]}")

In [ ]:
## Vendo quais variáveis são constantes ao longo do tempo (variância 0)
# Constantes: 
# agric(dummy se trabalha na agricultura), black e hisp (dummies de cor)
# manuf (dummy se trabalha na manufatura), south, nrthcen e nrtheast (dummies regionais)
# e, mais importante, educação (todos os homens são adultos e apenas trabalham)
df.groupby('nr').var()

### Montando a estrutura de painel

In [ ]:
## Vendo grau de balancemaneto/desbalanceamento
df['aparicoes'] = df.groupby('nr')['nr'].transform('count')
df.drop_duplicates(subset=['nr'], keep='first')['aparicoes'].value_counts(normalize=True)*100
# O painel é completamente balanceado (muito díficil na vida real, vide PNAD COVID e a PNADC, que costuma ser pior)

In [ ]:
## Montando a estrutura do painel
# drop=false mantém as colunas no DataFrame, o que facilita a inclusão de dummies anuais usando C(year)
df = df.set_index(["nr", "year"], drop=False)

In [ ]:
## Vendo o resultado
df.head(10)

### Modelos

Uma boa equação de salários para começar a análise de mudança no retorno da educação é 

lsalariohora = married + union + dt*educ

Por que?

- Pessoas casadas tendem a ganhar mais (seja porque se sentem mais motivadas, porque dividem o trabalho doméstico, várias teorias)

- Pessoas sindicalizadas tendem a ganhar mais e ter salários mais "pegajosos" (fenômeno mais particular dos EUA, principalmente antes das reformas dos anos 90)

In [ ]:
## Estimando a equação por efeitos fixos
# Operador ':' cria interações entre as variáveis
# Operador '*' cria interações entre as variáveis E as adiciona sozinhas
formula_simples = "lwage ~ 1 + union + married + C(year)*educ + EntityEffects"

# Criando o modelo
# drop_absorbed: se tiver alguma variável constante no tempo, pulá-la
modelo_ef_simples = PanelOLS.from_formula(formula=formula_simples, data=df, drop_absorbed=True)
modelo_ef_simples = modelo_ef_simples.fit(cov_type='robust')

# Printando resultado
print(modelo_ef_simples.summary)

A mensagem de erro acima nos diz que educ teve que ser *droppada* da equação, o que se deve a ela ser constante no tempo.

Só parece haver um aumento significante no retorno da educação (frente a 1980) a partir de 1986.
Em 1987, por exemplo, estima-se que o retorno educacional seja 3% maior do que era em 1980, o que pode ser explicado, por exemplo, pela revolução tecnológica e pela preferência a trabalhos mais intelectuais no setor de serviços.

Outra coisa a se notar é que não parece ter havido mudanças estruturais ao longo da década, o que é captado pelas dummies de cada ano (insignificantes individualmente).

In [ ]:
## Podemos testar se as variáveis dummies são conjuntamente significantes usando um teste F,
# que pode ser interpretado como uma versão do teste t aplicado a mais de uma variável
H0 = "0 = C(year)[T.1981] = C(year)[T.1982] = C(year)[T.1983] = C(year)[T.1984] = C(year)[T.1985] = C(year)[T.1986] = C(year)[T.1987]"
# Teste wald_test
print(f"O p-valor do teste é {modelo_ef_simples.wald_test(formula=H0).pval}")

# O p-valor é de 0,97, ou seja, não há mudanças estruturais no tempo (as mudanças temporais se devem a mudanças no retorno da educação)

### Comparando Modelos: Vale a pena ser sindicalizado?

Usaremos três métodos e uma equação mais completa para estimar a equação salarial de **homens**: MQO Agrupado, EF e EA

In [ ]:
## Definindo a fórmula
formula = "lwage ~ 1 + educ + black + hisp + exper + expersq + married + union + C(year)"
formula_ef = f"{formula} + EntityEffects"

## Criando os três modelos
# Lembrando: o tipo da covariância não afetas as estimativas centrais, apenas os erros-padrão e os testes t dos estimadores.
# Efeitos Fixos
modelo_ef = PanelOLS.from_formula(formula=formula_ef, data=df, drop_absorbed=True)
modelo_ef = modelo_ef.fit(cov_type='robust')

# Efeitos Aleatórios
modelo_ea = RandomEffects.from_formula(formula=formula, data=df).fit(cov_type='robust')

# OLS Agrupado
modelo_ols = PooledOLS.from_formula(formula=formula, data=df).fit(cov_type='robust')

In [ ]:
## Vendo os resultados
print("P-valores entre parênteses.")
print(compare(
    {'EF':modelo_ef, 
     'EA':modelo_ea,
     'MQOA':modelo_ols}, 
     precision = 'pvalues').summary)

Antes de iniciar a análise, observa-se que experiência também não é estimada em efeitos fixos.

Como incluímos dummies para todos os anos e todos os homens trabalham em todos os anos (ou seja, *exper* varia sempre em uma unidade), ela não pode ser estimada por EF (o aumento constante de um ano não pode ser distinguído do efeito temporal agregado).
expersq, por sua vez, muda de forma variável ao longo dos anos -- 20^2 - 19^2 é diferente de 21^2 - 20^2.

Quando controlamos para todos os efeitos não-observáveis (EF e EA), o retorno de ser sindicalizado e casado cai (o que indica que homens mais 'capazes' tendem a ser casados e mais sindicalizados).

Essa foi a conclusão de um estudo publicado na [*Journal of Applied Econometrics*](https://www.jstor.org/stable/223257): pessoas sindicalizadas tendem a ter uma heterogeneidade individual que contribui para os salários, o que pode ser visto nos maiores coeficientes de EA e MQOA.

In [ ]:
## Teste de Hausman para ver qual modelo é melhor
# Antes, precisamos garantir que EA tenha as MESMAS variáveis que EF,
# ou seja, sem as variáveis constantes no tempo
formula_ea = "lwage ~ 1 + expersq + married + union + C(year)"
modelo_ea = RandomEffects.from_formula(formula=formula_ea, data=df).fit(cov_type='kernel')

In [ ]:
## Calculando estatística de Hausman
# Variância Assintótica
var_assin = modelo_ef.cov - modelo_ea.cov

# Diferença entre os parâmetros
dif_parametros = modelo_ef.params - modelo_ea.params

# Calculando a estatística de Hausman
H = dif_parametros.dot(np.linalg.inv(var_assin)).dot(dif_parametros)

# Grau de Liberdade
graus_liberdade = modelo_ea.params.size - 1

# Calculando p-valor
# H0: não há correlação entre as variáveis independentes e as características não-observáveis
# Não-rejeitar H0: prefere-se EA, mas EF ainda é consisteente
# Rejeitar: prefere-se EF e EA é inconsistente
p = stats.chi2(graus_liberdade).sf(H)

print(p)

# As características não-observáveis NÃO são correlacionadas com as variáveis do modelo
# Assim, prefere-se EA 
# Melhor dos mundos: temos um estimador que elimina efeitos não-observados, consistente, eficiente 
# e que consegue estimar variáveis constantes no tempo

### Analisando as mudanças na sindicalização ao longo do tempo por Efeitos Aleatórios

In [ ]:
## Se a média de salário de sindicalizados é maior, espera-se que mais pessoas se tornem sindicalizadas
# Isso aconteceu mesmo?
df['year_column'] = df['year']  # criando para evitar um erro de ambiguidade no groupby (index e coluna se chamam year)
df.groupby('year_column')['union'].mean()

## Ao longo de 1980 a 1986, a sindicalização diminuiu ao longo do tempo

In [ ]:
## Vamos ver se o retorno da sindicalização mudou ao longo do tempo
# Novamente, usamos o operador *
formula_union = "lwage ~ 1 + educ + black + hisp + exper + expersq + married + union*C(year)"

# Criando o modelo
modelo_ea_union = RandomEffects.from_formula(formula=formula_union, data=df)
modelo_ea_union = modelo_ea_union.fit(cov_type='robust')

# Printando modelo_ef_union
print(modelo_ea_union.summary)

Como *union* não é constante no tempo, conseguimos estimá-la para o ano base.
Em 1980, o retorno para o salário de ser sindicalizado era de ≈+17,47%, o que se reduziu principalmente a partir de 1986 (mesmo ano em que a mudança no retorno da educação passa a ser significante no modelo anterior).

# **Aplicação 2**

In [ ]:
## Lendo o dataset
# Descrição das variáveis: https://rdrr.io/cran/wooldridge/man/driving.html
sArquivo = f"{sCaminho}driving.dta"
df = pd.read_stata(sArquivo)

In [ ]:
df.head()

In [ ]:
# Vendo o tamanho da base
print(f"Linhas: {df.shape[0]}; Colunas: {df.shape[1]}")

In [ ]:
## Descrevendo o dataset
df.describe()

# bac08 e bac10: limite máximo de álcool no sangue
# per se: revogação de licença imediatamente após crise de trânsito
# sbprim: oficiais de trânsito podem multar pessoas que não usem cinto de segurança (primary seatbelt law)
# sbsecon: oficiais de trânsito só podem multar a falta de cinto se houver algum outro motivo para parar o carro
# s170plus: limite de velocidade (speed limit) médio acima de 70 milhas por hora
# gdl: programa de supervisão de motoristas mais jovens
# perc14_24: percentagem da população entre 14 e 24 anos
# unem: taxa de desemprego
# vehicmilespc: número de milhas dirigidas por ano

Estamos interessados na variável *totfatrte* (*total fatality rate*): a taxa de fatalidade por 100.000 habitantes.

In [ ]:
## Vendo como ela se comportou ao longo do tempo
taxa_fatalidade_ano = df.groupby('year')['totfatrte'].mean()
taxa_fatalidade_ano.plot(kind="bar")
print(taxa_fatalidade_ano)

# A média nacional caiu: houve menos mortes no trânsito por 100.000 habitantes

### Estruturando o Painel e Modelo Inicial

In [ ]:
## Vendo grau de balancemaneto/desbalanceamento
df['aparicoes'] = df.groupby('state')['state'].transform('count')
df.drop_duplicates(subset=['state'], keep='first')['aparicoes'].value_counts(normalize=True)*100

# O painel é completamente balanceado, com todos os indivíduos sendo observados por 25 anos.
# (o que faz sentido, já que os dados são de estados e não indivíduos, que podem deixar de responder)

25    100.0
Name: aparicoes, dtype: float64

In [ ]:
## Montando a estrutura do painel
df = df.set_index(["state", "year"], drop=False)

In [ ]:
## Fazendo a mesma análise do gráfico usando MQO Agrupado
formula_ano = "totfatrte ~ 1 + C(year)"

# OLS Agrupado
modelo_ols_ano = PooledOLS.from_formula(formula=formula_ano, data=df).fit()
print(modelo_ols_ano.summary)

# Cada dummy possui um coeficiente menor que o do ano anterior (de modo geral)
# Como visto no gráfico, a taxa de fatalidade por 100.000 habitantes diminui ao longo do tempo
# A pergunta que fica é: por que?

### Comparando Modelos

In [ ]:
## Adicionando outros controles
formula = "totfatrte ~ 1+ C(year) + bac08 + bac10 + perse + sbprim + sbsecon + sl70plus + gdl + perc14_24 + unem + vehicmilespc"
# bac08 e bac10: limite máximo de álcool no sangue
# per se: revogação de licença imediatamente após crise de trânsito
# sbprim: oficiais de trânsito podem multar pessoas que não usem cinto de segurança (primary seatbelt law)
# sbsecon: oficiais de trânsito só podem multar a falta de cinto se houver algum outro motivo para parar o carro
# s170plus: limite de velocidade (speed limit) médio acima de 70 milhas por hora
# gdl: programa de supervisão de motoristas mais jovens
# perc14_24: percentagem da população entre 14 e 24 anos
# unem: taxa de desemprego
# vehicmilespc: número de milhas dirigidas por ano


# OLS Agrupado
modelo_ols = PooledOLS.from_formula(formula=formula, data=df).fit(cov_type="robust")  # controlando para heteroscedasticidade
print(modelo_ols.summary)

- Efeitos temporais se intensificaram, o que mostra que as mortes podem estar concentrados em estados sem leis mais fortes
- Limite de álcool no sangue mais severo contribui para a queda nas mortes de trânsito (-2.49 a cada 100.000 mil habitantes), bem como leis *per se*, apesar de menos intencidade
- Cinto de segurança parece não ter efeito sobre a mortalidade (p-valor alto)
- Limites maiores de velocidade estão ligados a 3.34 mais mortes anuais no trânsito por 100.000 habitantes;
- Motoristas e população jovem não tem efeito sobre a taxa de fatalidade
- Mais desemprego contribui para mais mortes no trânsito (por que)?
- Milhas dirigidas não parece estar ligado a mais mortes (apesar ter p-valor 0, o valor da estimativa é muito diminuto)

In [ ]:
## Regredindo por efeitos fixos
formula_ef = f"{formula} + EntityEffects"
modelo_ef = PanelOLS.from_formula(formula=formula_ef, data=df).fit(cov_type="robust")  # controlando para heteroscedasticidade

## Comparando os dois modelos
print("P-valores entre parênteses.")
print(compare(
    {'EF':modelo_ef, 
    'MQOA':modelo_ols}, 
    precision = 'pvalues').summary)

- Limites de álcool no sangue tem sua importância diminuída;
- Leis de cinto de segurança (primárias) diminuem a mortalidade
- Maiores limites de velocidade se tornam estatisticamente insignificantes
- Uma população mais jovem passa a ser marginalmente significante em explicar maiores taxas de fatalidade
- Desemprego passa a diminuir as mortes de trânsito e milhas dirigidas continuam tendo efeito diminuto

In [ ]:
## Fazendo Efeitos Aleatórios e comparando as estimativas
modelo_ea = RandomEffects.from_formula(formula=formula, data=df).fit(cov_type="robust")  # controlando para heteroscedasticidade

## Comparando os dois modelos
print("P-valores entre parênteses.")
print(compare(
    {'EA':modelo_ea,
    'EF':modelo_ef, 
    'MQOA':modelo_ols}, 
    precision = 'pvalues').summary)

In [ ]:
## As estimativas de EA parecem bem semelhantes às de EF: qual modelo é mais eficiente?
# Curiosidade: por ser calculado com uma quasi-média, EA SEMPRE SEMPRE SEMPRE estará entre as estimativas de MQOA e EF
# (quando todos os modelos estimarem as mesmas variáveis)

## Fazendo o teste de Hausman
# Variância Assintótica
var_assin = modelo_ef.cov - modelo_ea.cov

# Diferença entre os parâmetros
dif_parametros = modelo_ef.params - modelo_ea.params

# Calculando a estatística de Hausman
H = dif_parametros.dot(np.linalg.inv(var_assin)).dot(dif_parametros)

# Grau de Liberdade
graus_liberdade = modelo_ea.params.size - 1

# Calculando p-valor
# H0: não há correlação entre as variáveis independentes e as características não-observáveis
# Não-rejeitar H0: prefere-se EA, mas EF ainda é consisteente
# Rejeitar: prefere-se EF e EA é inconsistente
p = stats.chi2(graus_liberdade).sf(H)

print(p)

# As características não-observáveis NÃO são correlacionadas com as variáveis do modelo.
# Isso faz sentido, já que estamos tratando de estados e não pessoas
# Assim, prefere-se EA

# **Aplicação 3**



In [ ]:
## Lendo o dataset
# Descrição das variáveis: https://rdrr.io/cran/wooldridge/man/airfare.html
sArquivo = f"{sCaminho}airfare.dta"
df = pd.read_stata(sArquivo)

In [ ]:
df.head()

In [ ]:
# Vendo o tamanho da base
print(f"Linhas: {df.shape[0]}; Colunas: {df.shape[1]}")

Linhas: 4596; Colunas: 14


In [ ]:
## Número de (rotas): 1.149
# Como há 4596 linhas e 1.149 indivíduos, já sabemos que o painel é totalmente balanceado (4596/1149 = 4)
df.groupby('id')['id'].count()

In [ ]:
## Descrevendo o dataset
df.describe()

# dist: distancia da rota, em milhas
# passen: número médio de passageiros por dia
# fare: preço médio da passagem (só de ida)
# bmktshr=concen: fração do mercado da maior operadora da linha (biggest market share)
#   ^medida de concentração de mercado!

In [ ]:
## Vendo como a tarifa média se comportou ao longo do tempo
tarifa_ano = df.groupby('year')['fare'].mean()
tarifa_ano.plot(kind="bar")
print(tarifa_ano)

In [ ]:
## Montando a estrutura do painel
df = df.set_index(["id", "year"], drop=False)

In [ ]:
## O que pode determinar o aumento de preço por ano?
## Modelo MQOA
formula = "lfare ~ 1 + concen + ldist + ldistsq + C(year)"

modelo_ols = PooledOLS.from_formula(formula=formula, data=df).fit(cov_type="robust")
print(modelo_ols.summary)

- Um aumento de 10% na concentração de mercado aumenta em 3,6% o preço médio de uma rota;
- O preço da passagem cresce de forma exponencial com a distância (mesmo comportamento de educação nas equações de salário).
Note que o efeito combinado de *ldist* só é menor que 0 quando *ldist* < 4.35 (0,1030*ldist* < 0,9016*ldistsq*) e, consequentemente, *ldist* < 80. O menor valor de distância no dataset é de 95 milhas, ou seja, sempre a distância contribui positivamente para o preço no *range* dos nossos dados.

**Detalhe téorico**: como *ldist* e *lfare* estão em log, diz-se que os coeficientes de *ldist* são as **elasticidades** e dizem a variação percentual do preço para uma variação de 1% na distância.

Como usamos termos quadráticos, permitimos que essa elasticidade seja variável: nossas estimativas indicam que, quanto maior a distância, um incremento adicional de 1% leva a um aumento maior no preço do que em menores distâncias (o que faz sentido: 1% de 200 milhas é maior que 1% de 100 milhas e os custos de linhas aéreas costumam ser calculados por milhas (ou quilômetros) no ar).

De forma similar, como *concen* é uma taxa percentual, ela também é a **elasticidade do preço com relação a concentração de mercado**: seu coeficiente mostra, para uma variação de 1% na concentração, qual será a variação do preço

In [ ]:
## Comparando modelos
## Definindo a fórmula
formula_ef = f"{formula} + EntityEffects"

## Criando os três modelos
# Efeitos Fixos (não estimam a distância porque ela é fixa para uma dada rota)
modelo_ef = PanelOLS.from_formula(formula=formula_ef, data=df, drop_absorbed=True)
modelo_ef = modelo_ef.fit(cov_type='robust')

# Efeitos Aleatórios
modelo_ea = RandomEffects.from_formula(formula=formula, data=df).fit(cov_type='robust')

In [ ]:
## Vendo os resultados
print("P-valores entre parênteses.")
print(compare(
    {'EF':modelo_ef, 
     'EA':modelo_ea,
     'MQOA':modelo_ols}, 
     precision = 'pvalues').summary)

Apesar de menor, as estimativas para o efeito da concentração de EF e EA (que retiram particularidades não-observadas de cada rota) ainda são bastante altas e significativas: há distorção de oligopólio/monopólio em qualquer um dos modelos.

In [ ]:
## Por fim, teste de Hausman
# Modelo EA sem ldist e ldistsq
formula_ea = "lfare ~ 1 + concen + C(year)"
modelo_ea_ldist = RandomEffects.from_formula(formula=formula_ea, data=df).fit(cov_type='robust')

# Variância Assintótica
var_assin = modelo_ef.cov - modelo_ea_ldist.cov

# Diferença entre os parâmetros
dif_parametros = modelo_ef.params - modelo_ea_ldist.params

# Calculando a estatística de Hausman
H = dif_parametros.dot(np.linalg.inv(var_assin)).dot(dif_parametros)

# Grau de Liberdade
graus_liberdade = modelo_ea_ldist.params.size - 1

# Calculando p-valor
# H0: não há correlação entre as variáveis independentes e as características não-observáveis
# Não-rejeitar H0: prefere-se EA, mas EF ainda é consisteente
# Rejeitar: prefere-se EF e EA é inconsistente
p = stats.chi2(graus_liberdade).sf(H)

print(p)

# As características não-observáveis NÃO são correlacionadas com as variáveis do modelo
# Assim, prefere-se EA e estimativa mais precisa da elasticidade da concentração é ≈ 0,2

# Bom sábado a todos!